# Web scraping

In [1]:
import pandas as pd
import numpy as np
#pip install ntscraper
from ntscraper import Nitter

In [2]:
scraper = Nitter()

Testing instances: 100%|██████████| 29/29 [00:59<00:00,  2.04s/it]


## (1) Get tweets
Now we can get the tweets

In [3]:
tweets_food_term=scraper.get_tweets("food",mode="term",number=1000)

15-Dec-23 10:20:32 - No instance specified, using random instance https://nitter.poast.org
15-Dec-23 10:20:39 - Current stats for food: 13 tweets, 0 threads...
15-Dec-23 10:20:43 - Current stats for food: 27 tweets, 0 threads...
15-Dec-23 10:20:48 - Current stats for food: 39 tweets, 0 threads...
15-Dec-23 10:20:52 - Current stats for food: 45 tweets, 0 threads...
15-Dec-23 10:20:56 - Current stats for food: 53 tweets, 0 threads...
15-Dec-23 10:21:01 - Current stats for food: 63 tweets, 0 threads...
15-Dec-23 10:21:05 - Current stats for food: 73 tweets, 0 threads...
15-Dec-23 10:21:10 - Current stats for food: 85 tweets, 0 threads...
15-Dec-23 10:21:14 - Current stats for food: 97 tweets, 0 threads...
15-Dec-23 10:21:18 - Current stats for food: 101 tweets, 0 threads...
15-Dec-23 10:21:23 - Current stats for food: 110 tweets, 0 threads...
15-Dec-23 10:21:27 - Current stats for food: 118 tweets, 0 threads...
15-Dec-23 10:21:32 - Current stats for food: 122 tweets, 0 threads...
15-Dec-2

In [4]:
len(tweets_food_term) #2

2

### (1a) Small trials to discover tweets structure

In [124]:
print(type(tweets_food_hashtag))
for k in tweets_food_hashtag:
    print(k)
#type(tweets_food_hashtag["tweets"])

<class 'dict'>
tweets
threads


Threads is an empty list, since tweets seems interesting!

In [5]:
type(tweets_food_term["tweets"])

list

It is a list...inspect its elements

In [6]:
tweets_food_term["tweets"][2]

{'link': 'https://twitter.com/Christoo1972/status/1735590654275641745#m',
 'text': 'Boss please we need food stuffs badly I beg u. 8037565099 Opay Christopher Onochie. #Drbillsxmasgiveaway',
 'user': {'name': 'Chrstoo',
  'username': '@Christoo1972',
  'profile_id': '1699130734143746048',
  'avatar': 'https://pbs.twimg.com/profile_images/1699130734143746048/Y2AhjMDL_bigger.jpg'},
 'date': 'Dec 15, 2023 · 9:20 AM UTC',
 'is-retweet': False,
 'external-link': '',
 'quoted-post': {},
 'stats': {'comments': 0, 'retweets': 0, 'quotes': 0, 'likes': 0},
 'pictures': [],
 'videos': [],
 'gifs': []}

### (1b) Create a dataframe of tweets (and their info) 

In [9]:
type(tweets_food_term)

dict

In [10]:
final_tweets=[]
for tweet in tweets_food_term['tweets']:
    useful_data=[ #tweet text and username
                  tweet['text'],
                  tweet['user']['username'],
                  #stats of the tweet
                  tweet['stats']['likes'],
                  tweet['stats']['comments'],
                  tweet['stats']['retweets'],
                  tweet['stats']['quotes'],
                  tweet['is-retweet'],
                  tweet['external-link'], #useful: an external link might have a bad influence on eng rate!
                  #maybe these vars will be encoded in a has/has not binary label
                  tweet['pictures'], 
                  tweet['videos'],
                  tweet['gifs']                 
                ]       
    final_tweets.append(useful_data)

In [11]:
tweets_df= pd.DataFrame(final_tweets,columns= ['text','username','likes','comments','retweets','quotes','is-retweet','external-link','pictures','videos','gifs'])
print("Df dims: ",tweets_df.shape)

Df dims:  (1000, 11)


# Export dataset to CSV

In [12]:
# Remove duplicates
unique_tweets=tweets_df.drop_duplicates(subset=['text','username','likes','comments','retweets','quotes','is-retweet','external-link'],
                                        keep='first')

In [13]:
print("Df dims: ",unique_tweets.shape)

Df dims:  (989, 11)


In [15]:
dati_vecchi= pd.read_csv('../../datasets/tweets-data-giu.csv')
print("Df dims: ",dati_vecchi.shape)

Df dims:  (152, 11)


In [16]:
dati_nuovi= pd.concat([dati_vecchi,unique_tweets],ignore_index=True)

In [18]:
print("Df dims: ",dati_nuovi.shape)
dati_nuovi=dati_nuovi.drop_duplicates(subset=['text','username','likes','comments','retweets','quotes','is-retweet','external-link'],
                                        keep='first')
print("Df nuovo: ",dati_nuovi.shape)

Df dims:  (1141, 11)
Df nuovo:  (1141, 11)


In [19]:
dati_nuovi.to_csv('../../datasets/tweets-data-giu.csv', index=False)